In [26]:
import sqlite3 as lite
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import unicodedata
import re
# import pickle
import time
import sqlalchemy
from sqlalchemy import create_engine
import matplotlib as plt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.downloader.download('vader_lexicon')
nltk.downloader.download('punkt')

[nltk_data] Downloading package vader_lexicon to C:\Users\Evan Mc
[nltk_data]     Garry\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Evan Mc
[nltk_data]     Garry\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [8]:
# Import the 10-K Data from the Database
db_name = "SEC_Filings.db"
table_name = "10K_Data"

engine = sqlalchemy.create_engine('sqlite:///' + db_name, execution_options={"sqlite_raw_colnames": True})
df = pd.read_sql_table(table_name, engine)

# Clean DataFrame
df = df.drop(['Filing_Date', 'Filing_Link', 'Accession_No'], axis=1)
df

,SIC,CIK,Company,Filing_Document_Text
0,6029,0001527383,BankGuam Holding Co,['false 2021 fy 0001527383 --12-31 true p1y p1...
1,6029,0001527383,BankGuam Holding Co,['10-k 1 bkgm-10k_20201231.htm 10-k bkgm-10k_2...
2,6029,0001527383,BankGuam Holding Co,['10-k 1 bkgm-10k_20191231.htm 10-k bkgm-10k_2...
3,6029,0001527383,BankGuam Holding Co,['10-k 1 bkgmf-10k_20181231.htm 10-k bkgmf-10k...
4,6029,0001527383,BankGuam Holding Co,['10-k 1 bkgmf-10k_20171231.htm 10-k bkgmf-10k...
...,...,...,...,...
3912,0900,0001488419,"Sino Agro Food, Inc.",['10-k 1 tm2120163d1_10k.htm form 10-k united ...
3913,0900,0001488419,"Sino Agro Food, Inc.",['10-k 1 tm214645d1_10k.htm form 10-k united s...
3914,0900,0001488419,"Sino Agro Food, Inc.",['10-k 1 tv517282_10k.htm form 10-k united sta...
3915,0900,0001488419,"Sino Agro Food, Inc.",['10-k 1 tv488235_10k.htm form 10-k united sta...


In [46]:
# Isolate Top 5 Companies by Word Count
# Company 1: Upstart Holdings, Inc.
c1 = df.loc[df['Company'] == 'Upstart Holdings, Inc.']
c1_ri = c1.reset_index(drop=True)

# Company 2: Blackstone Inc.
c2 = df.loc[df['Company'] == 'Blackstone Inc.']
c2_ri = c2.reset_index(drop=True)

# Company 3: OppFi Inc.
c3 = df.loc[df['Company'] == 'OppFi Inc.']
c3_ri = c3.reset_index(drop=True)

# Company 4: Cosmos Group Holdings Inc.
c4 = df.loc[df['Company'] == 'Cosmos Group Holdings Inc.']
c4_ri = c4.reset_index(drop=True)

# Company 5: Riot Platforms, Inc.
c5 = df.loc[df['Company'] == 'Riot Platforms, Inc.']
c5_ri = c5.reset_index(drop=True)


,SIC,CIK,Company,Filing_Document_Text
0,6200,0001647639,"Upstart Holdings, Inc.","[""false 2022 fy 0001647639 http://fasb.org/us-..."
1,6200,0001647639,"Upstart Holdings, Inc.",['false 2021 fy 0001647639 12.50 .003506 00016...
2,6200,0001647639,"Upstart Holdings, Inc.",['10-k 1 upstrtholdingsinc202010-k.htm 10-k do...


In [56]:
# Sentiment Analysis
# Set 'Aspect' Words
word_search_list_2 = [' ai ', 'deep learning', 'deep-learning', 'computer vision', 'computer-vision', 'bots', 'automation', 'machine learning', 'machine-learning', 'algorithm', 'artificial intelligence', 'artificial-intelligence']
aspects = word_search_list_2

# Convert 10-K Text to String
review_text = c1_ri['Filing_Document_Text'][0]

# State Sentiment Analyzer - Rule-based Approach
sia = SentimentIntensityAnalyzer()

# Create Dictionary to Store Sentiment Scores
aspect_sentiments = {aspect: {'count': 0, 'sentiment': 0} for aspect in aspects}

# Perform Sentiment Analysis
sentences = sent_tokenize(review_text)
for sentence in sentences:
    for aspect in aspects:
        if aspect in sentence.lower():
            sent_score = sia.polarity_scores(sentence)
            aspect_sentiments[aspect]['count'] += 1
            aspect_sentiments[aspect]['sentiment'] += sent_score['compound']

# Aggregate Sentiment Scores
for aspect in aspect_sentiments:
    if aspect_sentiments[aspect]['count'] > 0:
        agg_sent = aspect_sentiments[aspect]['sentiment'] / aspect_sentiments[aspect]['count']
        print(f"Aggregated Sentiment for '{aspect}': {agg_sent}")

Aggregated Sentiment for ' ai ': 0.2941607407407407
Aggregated Sentiment for 'automation': 0.28095
Aggregated Sentiment for 'machine learning': 0.47458125
Aggregated Sentiment for 'algorithm': 0.36455
Aggregated Sentiment for 'artificial intelligence': 0.6081615384615385
